In [72]:
import os
import requests
from bs4 import BeautifulSoup
from datetime import datetime

class ReportScraper:
    def __init__(self, broker_code: int, report_type: str, start_date: str, end_date: str) -> None:
        self.url = "https://finance.naver.com/research"
        self.broker_code = broker_code  # 예: 신한투자증권(21)
        self.report_type = report_type + ".naver"  # 시황정보(market_info_list)

        # 날짜 범위 설정
        self.start_date = start_date
        self.end_date = end_date

    def _convert_to_date(self, date_str: str) -> datetime.date:
        """Convert a date string to a datetime.date object."""
        date_format = "%Y-%m-%d"
        return datetime.strptime(date_str, date_format).date()
    
    def collect_reports(self) -> list:
        """Collect reports within the set date range and return the report list."""
        # 보고서 리스트 수집
        report_list = self._get_report_list()
        return report_list

    def _get_report_list(self) -> list:
        """Collect PDF metadata within the set date range."""
        base_url = f"{self.url}/{self.report_type}?&searchType=brokerCode&brokerCode={self.broker_code}"
        page = 1
        metadata = []

        while True:
            # URL에 페이지 번호 추가
            url = f"{base_url}&page={page}"
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            table = soup.find('table')  # 해당 테이블의 class 이름에 따라 조정 가능

            # 테이블 내 각 행을 순회
            for row in table.find_all('tr'):
                cells = row.find_all('td')
                if len(cells) == 5:  # 필요한 데이터가 있는 행만 처리
                    title = cells[0].text.strip()
                    broker = cells[1].text.strip()
                    file_link = cells[2].find('a')['href'] if cells[2].find('a') else None
                    date = cells[3].text.strip()
                    views = cells[4].text.strip()
                    
                    # 스크래핑된 날짜를 YYYY-MM-DD 형식으로 변환
                    current_date = self.convert_to_long_format(date)

                    # 최신 날짜(start_date) 이후의 데이터는 스킵
                    if self.start_date < current_date:
                        continue
                    # 종료 날짜(end_date) 이전의 데이터는 수집 중단
                    if self.end_date > current_date:
                        return metadata
                    
                    # 데이터를 딕셔너리로 저장 (날짜는 YYYY-MM-DD 형식으로 저장)
                    data = {
                        'title': title,
                        'broker': broker,
                        'file_link': file_link,
                        'date': current_date,  # 이미 YYYY-MM-DD 형식으로 변환됨
                        'views': views
                    }
                    
                    # 리스트에 추가
                    metadata.append(data)

            # 다음 페이지가 있는지 확인하고 없으면 루프 종료
            next_page = soup.find('td', class_='pgRR')
            if not next_page:
                break

            page += 1

        return metadata

    @staticmethod
    def convert_to_long_format(date_str: str) -> str:
        """Convert YY.MM.DD format to YYYY-MM-DD format."""
        return datetime.strptime(date_str, '%y.%m.%d').strftime('%Y-%m-%d')

    def _download_pdf(self, metadata: dict, save_folder: str) -> None:
        """Download a PDF from the metadata and save it to the folder."""
        # 날짜와 제목을 기반으로 파일명 생성
        date = metadata['date']
        title = self._clean_filename(metadata['title'])  # 제목에 불법 문자 정리

        # 파일명 형식: {날짜}_{제목}.pdf
        filename = f"{date}_{title}.pdf"
        file_path = os.path.join(save_folder, filename)

        # PDF 파일 다운로드
        pdf_url = metadata['file_link']
        response = requests.get(pdf_url)

        # 응답이 성공적이면 파일 저장
        if response.status_code == 200:
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print(f"파일 다운로드 완료: {file_path}")
        else:
            print(f"파일 다운로드 실패: {pdf_url}")

    def _clean_filename(self, filename: str) -> str:
        """Replace illegal characters in the filename."""
        return filename.replace("/", "_").replace("\\", "_")

    def download_reports(self, report_list: list, save_folder: str) -> None:
        """Download each report in the provided report list."""
        # 저장할 폴더 생성
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)

        # 각 보고서를 다운로드
        for metadata in report_list:
            self._download_pdf(metadata, save_folder)

# 클래스 사용 예시
scraper = ReportScraper(broker_code=21, report_type="market_info_list", start_date="2024-09-27", end_date="2024-09-20")  # 날짜 범위와 함께 초기화
report_list = scraper.collect_reports()  # 보고서 수집
scraper.download_reports(report_list, save_folder="pdf_files")  # 보고서 다운로드


파일 다운로드 완료: pdf_files\2024-09-27_Global Platform Weekly(9월 4주차).pdf
파일 다운로드 완료: pdf_files\2024-09-27_Global Consumer Weekly(9월 4주차).pdf
파일 다운로드 완료: pdf_files\2024-09-27_마켓 레이더 (9월 27일, 오전).pdf
파일 다운로드 완료: pdf_files\2024-09-27_Global Daily (9월 27일).pdf
파일 다운로드 완료: pdf_files\2024-09-27_Global 헬스케어 Weekly(9월 4주차).pdf
파일 다운로드 완료: pdf_files\2024-09-27_Global Japan Weekly (9월 4주차).pdf
파일 다운로드 완료: pdf_files\2024-09-27_Global IT H_W Weekly (9월 4주차).pdf
파일 다운로드 완료: pdf_files\2024-09-27_국내 주식 마감 시황 - 9월 26일.pdf
파일 다운로드 완료: pdf_files\2024-09-26_Global 신재생 Weekly (9월 4주차).pdf
파일 다운로드 완료: pdf_files\2024-09-26_마켓레이더(9월 26일, 오전).pdf
파일 다운로드 완료: pdf_files\2024-09-26_Global Daily (9월 26일).pdf
파일 다운로드 완료: pdf_files\2024-09-26_국내 주식 마감 시황 - 9월 25일.pdf
파일 다운로드 완료: pdf_files\2024-09-25_마켓레이더(9월 25일, 오전).pdf
파일 다운로드 완료: pdf_files\2024-09-25_Global Daily (9월 25일).pdf
파일 다운로드 완료: pdf_files\2024-09-25_국내 주식 마감 시황 - 9월 24일.pdf
파일 다운로드 완료: pdf_files\2024-09-24_마켓레이더(9월 24일, 오전).pdf
파일 다운로드 완료: pdf_files\2024-09-